In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import re
import nltk
import math
import pickle

%matplotlib inline

In [2]:
# data downloaded from https://liveproject.manning.com/module/250_2_1/data-manipulation-and-arima-modeling-with-pyramid/1--data-manipulation%3a-forming-a-weekly-time-series/1-1-workflow%3a-data-manipulation%3a-forming-a-weekly-time-series?
df = pd.read_csv('H1.csv')

In [3]:
df.head()

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,...,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate
0,0,342,2015,July,27,1,0,0,2,0,...,No Deposit,NULL,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,0,737,2015,July,27,1,0,0,2,0,...,No Deposit,NULL,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,0,7,2015,July,27,1,0,1,1,0,...,No Deposit,NULL,NULL,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,0,13,2015,July,27,1,0,1,1,0,...,No Deposit,304,NULL,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,0,14,2015,July,27,1,0,2,2,0,...,No Deposit,240,NULL,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [4]:
df.dtypes

IsCanceled                       int64
LeadTime                         int64
ArrivalDateYear                  int64
ArrivalDateMonth                object
ArrivalDateWeekNumber            int64
ArrivalDateDayOfMonth            int64
StaysInWeekendNights             int64
StaysInWeekNights                int64
Adults                           int64
Children                         int64
Babies                           int64
Meal                            object
Country                         object
MarketSegment                   object
DistributionChannel             object
IsRepeatedGuest                  int64
PreviousCancellations            int64
PreviousBookingsNotCanceled      int64
ReservedRoomType                object
AssignedRoomType                object
BookingChanges                   int64
DepositType                     object
Agent                           object
Company                         object
DaysInWaitingList                int64
CustomerType             

In [5]:
df.columns

Index(['IsCanceled', 'LeadTime', 'ArrivalDateYear', 'ArrivalDateMonth',
       'ArrivalDateWeekNumber', 'ArrivalDateDayOfMonth',
       'StaysInWeekendNights', 'StaysInWeekNights', 'Adults', 'Children',
       'Babies', 'Meal', 'Country', 'MarketSegment', 'DistributionChannel',
       'IsRepeatedGuest', 'PreviousCancellations',
       'PreviousBookingsNotCanceled', 'ReservedRoomType', 'AssignedRoomType',
       'BookingChanges', 'DepositType', 'Agent', 'Company',
       'DaysInWaitingList', 'CustomerType', 'ADR', 'RequiredCarParkingSpaces',
       'TotalOfSpecialRequests', 'ReservationStatus', 'ReservationStatusDate'],
      dtype='object')

In [6]:
df.isna().sum()

IsCanceled                       0
LeadTime                         0
ArrivalDateYear                  0
ArrivalDateMonth                 0
ArrivalDateWeekNumber            0
ArrivalDateDayOfMonth            0
StaysInWeekendNights             0
StaysInWeekNights                0
Adults                           0
Children                         0
Babies                           0
Meal                             0
Country                        464
MarketSegment                    0
DistributionChannel              0
IsRepeatedGuest                  0
PreviousCancellations            0
PreviousBookingsNotCanceled      0
ReservedRoomType                 0
AssignedRoomType                 0
BookingChanges                   0
DepositType                      0
Agent                            0
Company                          0
DaysInWaitingList                0
CustomerType                     0
ADR                              0
RequiredCarParkingSpaces         0
TotalOfSpecialReques

## Load the dataset and use pandas to obtain the sum of cancellation entries by week. For example, form a new time series that contains the total number of cancellations per week.

In [7]:
df['ArrivalDateWeekNumber'].value_counts()

33    1197
31    1101
34    1090
30    1076
32    1054
29    1037
18    1012
28     988
35     951
43     917
15     906
21     906
7      904
41     857
22     849
17     840
13     811
27     810
9      795
38     774
20     769
19     762
36     756
16     755
23     752
39     748
14     739
10     727
44     726
42     724
26     721
8      719
49     716
47     708
25     704
45     690
6      682
11     679
40     675
24     675
12     656
37     633
53     618
52     592
3      553
4      519
46     514
5      501
48     493
2      457
50     449
51     430
1      343
Name: ArrivalDateWeekNumber, dtype: int64

In [8]:
print(sorted(df['ArrivalDateWeekNumber'].unique()))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]


In [9]:
df['IsCanceled'].value_counts()

0    28938
1    11122
Name: IsCanceled, dtype: int64

In [10]:
df['ArrivalDateYear'].value_counts()

2016    18567
2017    13179
2015     8314
Name: ArrivalDateYear, dtype: int64

In [11]:
#df['Date_Year_Week'] 
years = df['ArrivalDateYear'].values.astype(str)
weeks = df['ArrivalDateWeekNumber'].values.astype(str)

In [12]:
years[0:10]

array(['2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015',
       '2015', '2015'], dtype='<U21')

In [13]:
weeks[0:10]

array(['27', '27', '27', '27', '27', '27', '27', '27', '27', '27'],
      dtype='<U21')

In [14]:
print(len(years))
print()
print(len(weeks))

40060

40060


In [15]:
#df.head()
year_week = []

for item in zip(years,weeks):
    #print(item)
    #print(item[0]+item[1])
    year_week.append(item[0]+item[1])

In [16]:
df['Date_Year_and_Week_Number'] = year_week

In [17]:
df[['Date_Year_and_Week_Number','ArrivalDateYear','ArrivalDateWeekNumber']].head()

,Date_Year_and_Week_Number,ArrivalDateYear,ArrivalDateWeekNumber
0,201527,2015,27
1,201527,2015,27
2,201527,2015,27
3,201527,2015,27
4,201527,2015,27


In [18]:
df[['Date_Year_and_Week_Number','ArrivalDateYear','ArrivalDateWeekNumber']].tail()

,Date_Year_and_Week_Number,ArrivalDateYear,ArrivalDateWeekNumber
40055,201735,2017,35
40056,201735,2017,35
40057,201735,2017,35
40058,201735,2017,35
40059,201735,2017,35


In [19]:
df.groupby(['Date_Year_and_Week_Number'])['IsCanceled'].sum()

Date_Year_and_Week_Number
201527     41
201528     48
201529     87
201530     74
201531    101
         ... 
20175      51
20176      91
20177      96
20178     128
20179      77
Name: IsCanceled, Length: 115, dtype: int64

In [20]:
hug = df.groupby(['Date_Year_and_Week_Number'])['IsCanceled'].sum().values

In [21]:
hug = list(hug)

hug

[41,
 48,
 87,
 74,
 101,
 68,
 96,
 69,
 88,
 148,
 76,
 186,
 123,
 91,
 198,
 16,
 94,
 44,
 45,
 54,
 63,
 18,
 127,
 24,
 37,
 79,
 43,
 35,
 88,
 48,
 65,
 118,
 65,
 80,
 98,
 116,
 222,
 79,
 40,
 143,
 141,
 82,
 99,
 77,
 110,
 75,
 80,
 75,
 119,
 14,
 120,
 98,
 124,
 149,
 145,
 118,
 106,
 121,
 67,
 129,
 14,
 112,
 118,
 85,
 182,
 134,
 111,
 70,
 39,
 59,
 74,
 15,
 57,
 36,
 84,
 95,
 57,
 176,
 33,
 133,
 54,
 126,
 83,
 49,
 105,
 109,
 179,
 104,
 106,
 167,
 106,
 42,
 79,
 130,
 202,
 117,
 152,
 131,
 161,
 131,
 139,
 150,
 43,
 157,
 173,
 140,
 182,
 143,
 100,
 51,
 51,
 91,
 96,
 128,
 77]

In [22]:
df_hotel_cancel = df.groupby(['Date_Year_and_Week_Number'])['IsCanceled'].sum().reset_index()

In [23]:
df_hotel_cancel

,Date_Year_and_Week_Number,IsCanceled
0,201527,41
1,201528,48
2,201529,87
3,201530,74
4,201531,101
...,...,...
110,20175,51
111,20176,91
112,20177,96
113,20178,128


In [24]:
print(type(df_hotel_cancel))

<class 'pandas.core.frame.DataFrame'>


In [25]:
df_hotel_cancel.dtypes

Date_Year_and_Week_Number    object
IsCanceled                    int64
dtype: object

In [26]:
df_hotel_cancel.columns

Index(['Date_Year_and_Week_Number', 'IsCanceled'], dtype='object')

In [27]:
df_hotel_cancel.to_csv("hotel_cancellations.csv",index=False)

In [28]:
df_2 = pd.read_csv("hotel_cancellations.csv")

In [29]:
df_2.head()

,Date_Year_and_Week_Number,IsCanceled
0,201527,41
1,201528,48
2,201529,87
3,201530,74
4,201531,101
